In [15]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""
File: convert_tng_to_spam.py
Author: Matthew Ogden
Email: ogdenm12@gmail.com
Github: mbogden
Created: 2024-Apr-11

Description: This script is designed to take the dynamic kinematic data from the IllustrisTNG subhalo data, 
    and convert it into a format that can be used by the SPAM simulator. 

References:  Sections of this code were written  
    with the assistance of ChatGPT made by OpenAI.

"""
# ================================ IMPORTS ================================ #

import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

buildEnv = False


In [4]:
# ================================ CONSTANTS ================================ #

# Creating a dictionary to store all the SPAM units.  (From SPAM fortran source code)
spam_units = {
    "mass_gm": 1.98892e44,  # grams  (SPAM's mass unit, 10^11 M_⊙ )
    "mass_solar_gm": 1.98892e33,  # grams
    "distance_cm": 4.6285203749999994e22,  # cm  (SPAM's distance unit, roughly 15 kpc)
    "time_s": 2.733342473337471e15,  # s
    "velocity_cm/s": None,  # cm/s, to be calculated dynamically  (SPAM's velocity unit, roughly 170 km/s)
    "pc": 3.08568025e18,  # cm (parsec)
    "year": 365.25 * 24 * 3600,  # seconds in a year
    "km": 1e5,  # cm
    "kpc": None,  # kiloparsec, to be calculated dynamically
    "vel_km_sec": None,  # velocity in km/s, to be calculated dynamically
    "a_mss": None,  # Acceleration unit, to be calculated dynamically
    "a0_mks": 1.2e-10,  # Acceleration in m/s^2 (used in Modified Newtonian Dynamics)
    "a0": None,  # Normalized acceleration unit, to be calculated dynamically
    "pi": 3.141592653589793  # Pi
}

# Calculating dynamic values based on other constants
spam_units["velocity_cm/s"] = spam_units["distance_cm"] / spam_units["time_s"]
spam_units["kpc"] = spam_units["pc"] * 1000
spam_units["vel_km_sec"] = spam_units["velocity_cm/s"] / spam_units["km"]
spam_units["a_mss"] = spam_units["distance_cm"] / (spam_units["time_s"] ** 2) / 100.0
spam_units["a0"] = spam_units["a0_mks"] / spam_units["a_mss"]

# Dictionary of units for specific fields in the IllustrisTNG data
tng_units = {
    'mass_gm': 1.98892e+43,   # Mass unit equivalent to 10^10 solar masses in grams
    'distance_cm': 3.08568025e+21, # Comoving length unit equivalent to 1 kpc in cm per hubble parameter
    'velocity_cm/s': 100000.0,       # Velocity unit equivalent to 1 km/s in cm/s
}

#   Standardized parameter array for SPAM model:
'''
    [0]: X-coordinate of the secondary galaxy's position
    [1]: Y-coordinate of the secondary galaxy's position
    [2]: Z-coordinate of the secondary galaxy's position
    [3]: X-component of the secondary galaxy's velocity
    [4]: Y-component of the secondary galaxy's velocity
    [5]: Z-component of the secondary galaxy's velocity
    [6]: Mass of the primary galaxy
    [7]: Mass of the secondary galaxy
    [8]: Outer radius of the primary galaxy
    [9]: Outer radius of the secondary galaxy
    [10]: Azimuthal angle for the primary galaxy
    [11]: Azimuthal angle for the secondary galaxy
    [12]: Inclination angle for the primary galaxy
    [13]: Inclination angle for the secondary galaxy
    [14]: Softening length for the primary galaxy
    [15]: Softening length for the secondary galaxy
    [16]: Scaling factor for bulge in the primary galaxy
    [17]: Scaling factor for disk in the primary galaxy
    [18]: Scaling factor for halo in the primary galaxy
    [19]: Scaling factor for bulge in the secondary galaxy
    [20]: Scaling factor for disk in the secondary galaxy
    [21]: Scaling factor for halo in the secondary galaxy
'''


"\n    [0]: X-coordinate of the secondary galaxy's position\n    [1]: Y-coordinate of the secondary galaxy's position\n    [2]: Z-coordinate of the secondary galaxy's position\n    [3]: X-component of the secondary galaxy's velocity\n    [4]: Y-component of the secondary galaxy's velocity\n    [5]: Z-component of the secondary galaxy's velocity\n    [6]: Mass of the primary galaxy\n    [7]: Mass of the secondary galaxy\n    [8]: Outer radius of the primary galaxy\n    [9]: Outer radius of the secondary galaxy\n    [10]: Azimuthal angle for the primary galaxy\n    [11]: Azimuthal angle for the secondary galaxy\n    [12]: Inclination angle for the primary galaxy\n    [13]: Inclination angle for the secondary galaxy\n    [14]: Softening length for the primary galaxy\n    [15]: Softening length for the secondary galaxy\n    [16]: Scaling factor for bulge in the primary galaxy\n    [17]: Scaling factor for disk in the primary galaxy\n    [18]: Scaling factor for halo in the primary galaxy\n

In [6]:
# Testing on premade file
if buildEnv:
    
    # Specify the file path to illustrisTNG Target info
    file_path = '/home/mbo2d/galStuff/galaxyJSPAM/illustrisTNG_targets/tng-targets/my-target-list_dynamics.csv'

    # Read the CSV file into a Pandas DataFrame
    tng_targets_raw = pd.read_csv(file_path, on_bad_lines='warn')

    print( tng_targets_raw.columns )

Index(['moi_SubhaloIDRaw', 'snapnum', 'p_acceleration', 'p_SubhaloIDRaw',
       's_SubhaloIDRaw', 'p_SubhaloMass', 's_SubhaloMass', 'p_SubhaloPos',
       's_SubhaloPos', 'p_SubhaloVel', 's_SubhaloVel', 'p_SubhaloSpin',
       's_SubhaloSpin', 'p_SubhaloHalfmassRad', 's_SubhaloHalfmassRad',
       'xy_projection', 'p_face_projection', 's_face_projection',
       'pa_SubhaloMass', 'pa_SubhaloCM', 'pa_SubhaloVel', 'pa_SubhaloSpin',
       'pa_SubhaloHalfmassRad', 'sa_SubhaloMass', 'sa_SubhaloCM',
       'sa_SubhaloVel', 'sa_SubhaloSpin', 'sa_SubhaloHalfmassRad',
       'ps_SubhaloMass', 'ps_SubhaloCM', 'ps_SubhaloVel', 'ps_SubhaloSpin',
       'ps_SubhaloHalfmassRad', 'ss_SubhaloMass', 'ss_SubhaloCM',
       'ss_SubhaloVel', 'ss_SubhaloSpin', 'ss_SubhaloHalfmassRad'],
      dtype='object')


In [7]:
# ================================ FUNCTIONS ================================ #
def calculate_orientation_angles( spin ):
    """
    Calculate the azimuthal and inclination angles from the spin components.

    Parameters:
    spin (float array): [ x, y, z ] components of the spin.

    Returns:
    tuple: (phi, theta) where phi is the azimuthal angle and theta is the inclination angle,
            both in radians.
    """
    x, y, z = spin
    # Calculate the magnitude of the vector
    r = np.sqrt(x**2 + y**2 + z**2)

    # Calculate the azimuthal angle in radians
    phi = np.arctan2(y, x)  # This returns the angle in the range [-pi, pi]

    # Calculate the inclination angle in radians
    if r == 0:
        theta = 0  # Undefined direction for a zero vector, set arbitrarily to 0
    else:
        theta = np.arccos(z / r)

    return np.degrees(phi), np.degrees(theta)


def convert_numpy_arrays( df ):
    """
    Convert strings of arrays into numpy arrays in a pandas DataFrame.
    """

    # Function to convert a string representation of a numpy array back to a numpy array
    def convert_to_numpy_array(array_str):
        # Remove the brackets and split the string by whitespace
        array_str = array_str.strip('[]')
        # Convert to a numpy array of integers (or float if needed)
        return np.array(array_str.split(), dtype=float)
    
    # Iterate over each column and convert strings to numpy arrays if necessary
    for column in df.columns:
        # Check if the column contains array-like strings
        if df[column].dtype == object and df[column].str.startswith('[').all():
            df[column] = df[column].apply(convert_to_numpy_array)

    return df

if buildEnv:
    tng_targets = convert_numpy_arrays(tng_targets_raw)
    print( tng_targets.iloc[0])

moi_SubhaloIDRaw                                            67000000356635
snapnum                                                                 67
p_acceleration                                                    2.934985
p_SubhaloIDRaw                                              67000000356635
s_SubhaloIDRaw                                              67000000356638
p_SubhaloMass                                                   101.785866
s_SubhaloMass                                                     0.352484
p_SubhaloPos                             [28677.182, 26284.145, 22729.666]
s_SubhaloPos                             [28677.799, 26286.256, 22732.273]
p_SubhaloVel                              [20.427433, 96.62563, 65.574455]
s_SubhaloVel                             [-57.16416, 74.65875, -303.97495]
p_SubhaloSpin                           [-637.60767, -1269.724, 371.26877]
s_SubhaloSpin                         [-3.6678712, -26.946321, 0.62325656]
p_SubhaloHalfmassRad     

In [22]:
def standardize_tng_parameters_v1( subhalo_kinematics, ):
    """
    Construct the standardized parameter array for kinematic data from the IllustrisTNG subhalo catalog.
    Uses a mix of the catalog data, and calculated dynamics of all particles or star particles. 

    Args:
        subhalo_kinematics (pd.Series): The dynamic kinematic data calculated elsewhere.

    Returns:
        param_array (np.array): A standardized parameter array based on the SPAM simulator.
    """
    # Initialize the parameter array
    tng_ar = np.zeros(22)

    # position and velocity. 
    # Primary is always at origin, secondary is in reference to primary's location
    for i in range(3):
        tng_ar[i] = subhalo_kinematics['ss_SubhaloCM'][i] - subhalo_kinematics['ps_SubhaloCM'][i]  # position
        tng_ar[i+3] = subhalo_kinematics['ss_SubhaloVel'][i] - subhalo_kinematics['ps_SubhaloVel'][i]  # velocity

    # masses, using total mass
    tng_ar[6] = subhalo_kinematics['pa_SubhaloMass']
    tng_ar[7] = subhalo_kinematics['sa_SubhaloMass']
    
    # Radii, using stars to disk radii
    tng_ar[8] = subhalo_kinematics['ps_SubhaloHalfmassRad']
    tng_ar[9] = subhalo_kinematics['ss_SubhaloHalfmassRad']

    # orientation angles, using stars to orient disk
    tng_ar[10], tng_ar[12] = calculate_orientation_angles( subhalo_kinematics['ps_SubhaloSpin'] )
    tng_ar[11], tng_ar[13] = calculate_orientation_angles( subhalo_kinematics['ss_SubhaloSpin'] )

    # Just use a scale of previously established values
    s = 0.2
    tng_ar[14] = s * tng_ar[8]
    tng_ar[15] = s * tng_ar[9]

    # Use 0 for bulge, disk, and halo.  SPAM can estimate these values
    tng_ar[16:22] = np.zeros(6)    

    return tng_ar

if buildEnv:
    test_target = tng_targets.iloc[0]
    #print( test_target )
    test_tng_ar = standardize_tng_parameters_v1( test_target )
    print( test_tng_ar )


NameError: name 'buildEnv' is not defined

In [ ]:
def standardize_tng_parameters_v2( subhalo_kinematics, ):
    """
    Construct the standardized parameter array for kinematic data from the IllustrisTNG subhalo catalog.
    Uses a mix of the catalog data, and calculated dynamics of all particles or star particles. 

    Args:
        subhalo_kinematics (pd.Series): The dynamic kinematic data calculated elsewhere.

    Returns:
        param_array (np.array): A standardized parameter array based on the SPAM simulator.
    """
    # Initialize the parameter array
    tng_ar = np.zeros(22)

    # position and velocity. 
    # Primary is always at origin, secondary is in reference to primary's location
    for i in range(3):
        tng_ar[i] = subhalo_kinematics['s_SubhaloPos'][i] - subhalo_kinematics['p_SubhaloPos'][i]  # position
        tng_ar[i+3] = subhalo_kinematics['ss_SubhaloVel'][i] - subhalo_kinematics['ps_SubhaloVel'][i]  # velocity

    # masses, using total mass
    tng_ar[6] = subhalo_kinematics['pa_SubhaloMass']
    tng_ar[7] = subhalo_kinematics['sa_SubhaloMass']
    
    # Radii, using stars to disk radii
    tng_ar[8] = subhalo_kinematics['ps_SubhaloHalfmassRad']
    tng_ar[9] = subhalo_kinematics['ss_SubhaloHalfmassRad']

    # orientation angles, using stars to orient disk
    tng_ar[10], tng_ar[12] = calculate_orientation_angles( subhalo_kinematics['ps_SubhaloSpin'] )
    tng_ar[11], tng_ar[13] = calculate_orientation_angles( subhalo_kinematics['ss_SubhaloSpin'] )

    # Just use a scale of previously established values
    s = 0.2
    tng_ar[14] = s * tng_ar[8]
    tng_ar[15] = s * tng_ar[9]

    # Use 0 for bulge, disk, and halo.  SPAM can estimate these values
    tng_ar[16:22] = np.zeros(6)    

    return tng_ar

if buildEnv:
    test_target = tng_targets.iloc[0]
    #print( test_target )
    test_tng_ar = standardize_tng_parameters_v2( test_target )
    print( test_tng_ar )

In [16]:
# Conversion function for distances/positions
def convert_distance_tng_to_spam(tng_dist, h=0.7):
    # Convert ckpc/h to kpc
    dist_kpc = tng_dist * h
    # Convert kpc to cm
    dist_cm = dist_kpc * tng_units["distance_cm"]
    # Convert cm to SPAM distance units
    spam_dist = dist_cm / spam_units["distance_cm"]
    
    return spam_dist

# Conversion function for velocities
def convert_velocity_tng_to_spam(tng_vel):
    # Convert km/s to cm/s
    vel_cm_per_s = tng_vel * tng_units['velocity_cm/s']
    # Convert cm/s to SPAM velocity units
    spam_vel = vel_cm_per_s / spam_units["velocity_cm/s"]
    
    return spam_vel

# Conversion function for masses
def convert_mass_tng_to_spam(tng_mass, h=0.7):
    # Convert 10^10 M⊙/h to 10^10 M⊙
    mass_10e10_Msolar = tng_mass * h
    # Convert 10^10 M⊙ to grams
    mass_gm = mass_10e10_Msolar * tng_units['mass_gm']
    # Convert grams to SPAM mass units
    spam_mass = mass_gm / spam_units["mass_gm"]
    
    return spam_mass

# Function to convert the entire IllustrisTNG array to SPAM units
def convert_units_tng_to_spam_v2(tng_ar, h=0.7):
    """
    Convert IllustrisTNG data to SPAM units.   

    Parameters:
    - tng_ar: numpy array, array of targets in IllustrisTNG units.
    - h: float, the Hubble constant used in the IllustrisTNG simulation (dimensionless, usually ~0.7).

    Returns:
    - spam_data: numpy array, target converted to SPAM units.
    """
    
    # Initialize spam data array
    spam_data = np.zeros(22)

    # Convert Positions
    spam_data[0:3] = convert_distance_tng_to_spam(tng_ar[0:3], h)
    
    # Convert Velocities
    spam_data[3:6] = convert_velocity_tng_to_spam(tng_ar[3:6])
    
    # Convert Masses
    spam_data[6:8] = convert_mass_tng_to_spam(tng_ar[6:8], h)
    
    # Convert Radii
    spam_data[8:10] = convert_distance_tng_to_spam(tng_ar[8:10], h)
    spam_data[14:16] = spam_data[8:10] * 0.1  # From JSPAM paper. Softening length is typically 0.3 times outer radius

    # Use same orientation angles
    spam_data[10:14] = tng_ar[10:14]

    # Scaling factors are assumed 0 for now.

    return spam_data
# Example usage
if buildEnv:
    test_spam_ar = convert_units_tng_to_spam_v2(test_tng_ar)
    for i in range( len( test_spam_ar ) ):
        print( i, '\t', test_spam_ar[i] )


0 	 0.03142632266654498
1 	 0.18427225553326332
2 	 0.7931838781999251
3 	 -0.4556387669368036
4 	 -0.16413146001715034
5 	 -2.032632809675953
6 	 6.4503310666831775
7 	 0.6993532484778116
8 	 0.4517516774743448
9 	 0.6035078978424411
10 	 -165.4060118862749
11 	 -145.1485868676779
12 	 68.8817793359837
13 	 86.41717088126396
14 	 0.045175167747434485
15 	 0.06035078978424411
16 	 0.0
17 	 0.0
18 	 0.0
19 	 0.0
20 	 0.0
21 	 0.0


In [18]:
# Conversion function for distances/positions from SPAM to TNG units
def convert_distance_spam_to_tng(spam_dist, h=0.7):
    # Convert SPAM distance units to cm
    dist_cm = spam_dist * spam_units["distance_cm"]
    # Convert cm to kpc
    dist_kpc = dist_cm / tng_units["distance_cm"]
    # Convert kpc to ckpc/h
    tng_dist = dist_kpc / h
    
    return tng_dist

# Conversion function for velocities from SPAM to TNG units
def convert_velocity_spam_to_tng(spam_vel):
    # Convert SPAM velocity units to cm/s
    vel_cm_per_s = spam_vel * spam_units["velocity_cm/s"]
    # Convert cm/s to km/s
    tng_vel = vel_cm_per_s / tng_units["velocity_cm/s"]
    
    return tng_vel

# Conversion function for masses from SPAM to TNG units
def convert_mass_spam_to_tng(spam_mass, h=0.7):
    # Convert SPAM mass units to grams
    mass_gm = spam_mass * spam_units["mass_gm"]
    # Convert grams to 10^10 M⊙
    mass_10e10_Msolar = mass_gm / tng_units['mass_gm']
    # Convert 10^10 M⊙ to 10^10 M⊙/h
    tng_mass = mass_10e10_Msolar / h
    
    return tng_mass

# Function to convert the entire SPAM array to IllustrisTNG units
def convert_units_spam_to_illustris(spam_ar, h=0.7):
    """
    Convert SPAM data to IllustrisTNG units.   

    Parameters:
    - spam_ar: numpy array, array of targets in SPAM units.
    - primary_pos: numpy array, position of the primary galaxy in IllustrisTNG units (ckpc/h).
    - h: float, the Hubble constant used in the IllustrisTNG simulation (dimensionless, usually ~0.7).

    Returns:
    - tng_data: numpy array, target converted to IllustrisTNG units.
    """
    
    # Initialize tng data array
    tng_data = np.zeros(22)

    # Convert Positions (assuming secondary galaxy)
    tng_data[0:3] = convert_distance_spam_to_tng(spam_ar[0:3], h)
    
    # Convert Velocities
    tng_data[3:6] = convert_velocity_spam_to_tng(spam_ar[3:6])
    
    # Convert Masses
    tng_data[6:8] = convert_mass_spam_to_tng(spam_ar[6:8], h)
    
    # Convert Radii
    tng_data[8:10] = convert_distance_spam_to_tng(spam_ar[8:10], h)
    
    # Use same orientation angles
    tng_data[10:14] = spam_ar[10:14]
    
    # NOTE: Softening lengths and scales are not needed.
    

    return tng_data

# Example usage
if buildEnv:
    test_tng_ar_back = convert_units_spam_to_illustris(test_spam_ar)
    for i in range( len( test_tng_ar_back ) ):
        print( test_tng_ar[i], test_tng_ar_back[i] )


0.6734211999973923 0.6734211999973923
3.948691189998499 3.9486911899984993
16.99679738999839 16.99679738999839
-77.15583894 -77.15583894
-27.793290240000005 -27.793290240000008
-344.19698469 -344.19698469
92.14758666690253 92.14758666690253
9.990760692540166 9.990760692540166
9.68039308873596 9.680393088735958
12.932312096623736 12.932312096623736
-165.4060118862749 -165.4060118862749
-145.1485868676779 -145.1485868676779
68.8817793359837 68.8817793359837
86.41717088126396 86.41717088126396
1.936078617747192 0.0
2.5864624193247474 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0


In [ ]:
def generate_illustrisTNG_SPAM_targets_file( tng_file_loc, spam_file_loc ):

    # Assert file exists
    assert os.path.exists(tng_file_loc), f"File not found: {file_path}"

    # Read the CSV file into a Pandas DataFrame
    tng_targets = pd.read_csv(tng_file_loc, on_bad_lines='warn')

    # convert numpy arrays
    tng_targets = convert_numpy_arrays(tng_targets)

    # Create list to store spam info
    n = len(tng_targets)
    target_params = []

    # Loop through targets and create spam parameters
    for i in range(n):
        tmp_param = standardize_parameters_v1( tng_targets.iloc[i] )
        target_params.append( convert_units_illustris_to_spam( tmp_param ) )

    # Store list as new column
    tng_targets['spam_params_v1'] = target_params

    # Save to new file
    tng_targets.to_csv( spam_file_loc )

    return tng_targets

if buildEnv:
    tng_targets = generate_illustrisTNG_SPAM_targets_file( 
        tng_file_loc = '/home/mbo2d/galStuff/galaxyJSPAM/illustrisTNG_targets/tng-targets/my-target-list_dynamics.csv', 
        spam_file_loc = '/home/mbo2d/galStuff/galaxyJSPAM/illustrisTNG_targets/tng-targets/test_spam_targets.csv',
        )

